<a href="https://colab.research.google.com/github/roman-4erkasov/image_segmentation/blob/main/cv03_image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install seaborn

import torch
import numpy as np
import cv2
import torchvision
import PIL as pil
import seaborn as sns
import json


def show_image(image, figsize=(16, 9), reverse=True):
    plt.figure(figsize=figsize)
    if reverse:
        plt.imshow(image[...,::-1])
    else:
        plt.imshow(image)
    plt.axis('off')
    plt.show()

def get_device():
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(f"device={device}")
    return device

_ = get_device()

DEBUG = False

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
device=cpu


In [ ]:
to_tensor = torchvision.transforms.ToTensor()

def visualize_prediction(
    file, model, coco_id_to_name, device='cuda', verbose=True, thresh=0.0, n_colors=None
):
    img = pil.Image.open(file)
    img_tensor = to_tensor(img).unsqueeze(0)
    model.to(device)
    model.eval()
    with torch.no_grad():
        predictions = model(img_tensor.to(device)) # list of size 1
    prediction = predictions[0]
    
    if n_colors is None:
        n_colors = model.roi_heads.box_predictor.cls_score.out_features
    
    palette = sns.color_palette(None, n_colors)
    
    # visualize
    img = cv2.imread(file, cv2.COLOR_BGR2RGB)
    for i in range(len(prediction['boxes'])):
        x1, x2, x3, x4 = map(int, prediction['boxes'][i].tolist())
        label = int(prediction['labels'][i].cpu())
        score = float(prediction['scores'][i].cpu())
        name = coco_id_to_name[label]
        color = palette[label]
        if verbose:
            if score > thresh:
                print ('Class: {}, Confidence: {}'.format(name, score))
        if score > thresh:
            image = cv2.rectangle(img, (x1, x2), (x3, x4), np.array(color) * 255, 2)
            cv2.putText(image, name, (x1, x2-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, np.array(color) * 255, 2)
    show_image(image)
    return prediction


In [ ]:
#!g2.1

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    pretrained=True, 
#     progress=True,
#     num_classes=91, 
#     pretrained_backbone=True,
#     map_location=torch.device('cpu')
)


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /tmp/xdg_cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [ ]:
#!g2.1
device = get_device()
filepath = "/home/jupyter/mnt/s3/cv03-fasterrcnn-detection/detection_examples/mrs_maisel.jpg"
img = pil.Image.open(filepath)
img_tensor = to_tensor(img).unsqueeze(0)
# model.to(device)
model.eval()
with torch.no_grad():
    pred = model(img_tensor)[0]    
pred.keys()

device=cuda


/usr/local/lib/python3.7/dist-packages/torchvision/ops/boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


dict_keys(['boxes', 'labels', 'scores'])

In [ ]:
file = "/home/jupyter/mnt/s3/cv03-fasterrcnn-detection/detection_examples/mrs_maisel.jpg"
img = pil.Image.open(file)
img


In [ ]:

path = "/home/jupyter/mnt/s3/cv03-fasterrcnn-detection/coco_id_to_name.json"
with open(path, 'r') as f:
    coco_id_to_name = json.load(f)

out = visualize_prediction(
    file=file, 
    model=model, 
    coco_id_to_name=coco_id_to_name,
    device="cuda"
)

# out = visualize_prediction(file, model, device="cpu")



/usr/local/lib/python3.7/dist-packages/torchvision/ops/boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


KeyError: 1

In [ ]:
#!g1.1
